# Trampoline no name match

### Imports

In [74]:
import pandas as pd
import plotly.express as px
import os
import sqlite3
import hashlib
import matplotlib.pyplot as plt
import numpy as np
from decimal import Decimal

### Loading DATs

In [101]:
# dataFolder = "/Users/falkowork/Downloads/WCH Tokyo TRA Herren/Day 1 TRA Men/"
dataFolder = "/Users/falkowork/Downloads/Olympia 2021 Tokyo/SOLG_Tokyo_2020_GT_Men_EuroTrampMachines_Logs/"
db_name = "2020_olympic_games_men"
sortedList = sorted(os.listdir(dataFolder))
header = ('Start Number', "Gender", "Country", "Phase", "Routine", "Name", "Hash")
df_main = pd.DataFrame(columns=header)
connection = sqlite3.connect("trampoline.db")
cursor = connection.cursor()
df = pd.read_sql("SELECT * from ranklists", connection)
df_wctokyo = df[(df['Event']=='Olympic Games') & (df['Location']=='Tokyo') & (df['Gender']=='Men')]

In [102]:
routine = '0'
phase = '0'
t_list = []
mmc = 0
nmc = 0
gmc = 0
allc = 0
names = []
h_list = []
t_s = []
start_number = 0
for dataset in sortedList:
    if not ".DS_Store" in dataset:
        allc += 1
        # idx += 1
        # print(idx)
        datContent = [i.strip().split() for i in open(dataFolder + dataset).readlines()]
        trampoline_data = datContent[4:14]
        df_trampoline = pd.DataFrame(trampoline_data)
        if len(df_trampoline.columns) > 2:
            df_trampoline.drop([0, 1, 3], axis=1,inplace=True)
            header = ("T", "H", "x", "y")
            df_trampoline.columns = header
            sum_h = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['H'].to_list()])
            sum_t = np.sum([int(Decimal(i)*1000) if i is not None else 0 for i in df_trampoline['T'].to_list()])
            
            
            test = df_wctokyo[(df_wctokyo['H']*1000==sum_h) & (df_wctokyo['T']*1000==sum_t)]
            # if dataset == '20210731_135110.dat':
            print('H: {0}, T: {1}'.format(sum_h, sum_t))
            #     print(df_trampoline)
            if len(test) > 1:
                mmc += 1
                # print(dataset)
                # print(test['Name'])
            elif len(test) == 0:
                nmc += 1
                # print(dataset)
                # print("No Match")
                # print(sum_t)
            elif len(test) == 1:
                # if gmc == 0:
                #     print('H: {0}, T: {1}'.format(sum_h, sum_t))
                #     print(test)
                t_list.append(sum_t)
                h_list.append(sum_h)
                names.append(test["Name"].iloc[0])

                # creates hash from filename
                hash_val = hashlib.md5(dataset.encode('UTF-8')).hexdigest()
                df_trampoline.to_sql(name=hash_val, con=connection)

                # creates temporary df
                name = test["Name"].iloc[0].split()
                name = name[0].capitalize() + " " + name[1].capitalize()
                check_df = df_main[(df_main['Name']==name)]
                if len(check_df) == 0:
                    routine = '1st'
                    phase = 'Qualification'
                elif len(check_df) == 1:
                    routine = '2nd'
                    phase = 'Qualification'
                elif len(check_df) == 2:
                    routine = '1st'
                    phase = 'Final'
                d = {
                    'Start Number': start_number, 
                    "Gender": test["Gender"].iloc[0], 
                    "Country": test["Country"].iloc[0], 
                    "Phase": phase, 
                    "Routine": routine, 
                    "Name": name, 
                    "Hash": hash_val
                }
                df_temp = pd.DataFrame(d, index=[gmc])
                df_main = pd.concat([df_main, df_temp])
                gmc += 1


df_main.to_sql(name=db_name, con=connection)
connection.close()
print('Multi Match: {0}, No Match: {1}, All: {2}, Matched: {3}'.format(mmc, nmc, allc, gmc))

H: 4200, T: 7980
H: 2300, T: 3435
H: 4800, T: 8600
H: 5700, T: 10155
H: 3700, T: 6490
H: 4600, T: 8500
H: 2700, T: 5350
H: 2900, T: 5890
H: 7200, T: 14135
H: 9600, T: 18580
H: 9000, T: 17895
H: 9500, T: 15810
H: 9400, T: 18070
H: 9900, T: 17550
H: 9700, T: 18130
H: 4800, T: 8580
H: 4500, T: 8335
H: 3300, T: 6475
H: 3500, T: 10835
H: 9600, T: 16965
H: 7400, T: 12765
H: 3000, T: 4790
H: 2700, T: 4540
H: 9800, T: 17815
H: 9200, T: 19115
H: 9700, T: 17805
H: 9500, T: 18435
H: 9000, T: 18065
H: 9700, T: 17785
H: 9800, T: 17875
H: 9600, T: 18745
H: 1000, T: 1890
H: 3400, T: 6915
H: 9400, T: 17420
H: 9300, T: 16690
H: 9000, T: 14980
H: 9000, T: 16675
H: 8600, T: 15780
H: 9100, T: 17350
H: 9500, T: 16805
H: 8900, T: 17640
H: 9400, T: 17335
H: 6500, T: 12285
H: 4600, T: 8885
H: 6400, T: 12085
H: 9200, T: 16675
H: 1500, T: 3575
H: 1800, T: 3410
H: 1600, T: 2795
H: 1800, T: 3375
H: 3500, T: 5940
H: 2800, T: 4600
H: 2700, T: 4380
H: 900, T: 1365
H: 8900, T: 17100
H: 8500, T: 16735
H: 8700, T: 1691

In [103]:
df_main

,Start Number,Gender,Country,Phase,Routine,Name,Hash
0,0,Men,JPN,Qualification,1st,Sakai Ryosuke,b629b8eb8607ffede9ff29fca1654744
1,0,Men,BLR,Qualification,1st,Hancharou Uladzislau,46f31958238d1fa2acc6b383a93e40d6
2,0,Men,ROC,Qualification,1st,Ushakov Dmitrii,704bd97396839d13b1df4ad75ecea263
3,0,Men,EGY,Qualification,1st,Sherif Seif,ca96327bc2ef93da020465cb21cbc781
4,0,Men,ROC,Qualification,1st,Yudin Andrey,f04c5a3e1bbce2e2bd64a1129e75eac5
5,0,Men,COL,Qualification,1st,Hernandez Angel,e112e96a25f11654d12693bc5ab31f65
6,0,Men,AUS,Qualification,1st,Clarke Dominic,349c75d121405de30f7ec025a88db24a
7,0,Men,NZL,Qualification,1st,Schmidt Dylan,4c14201f63d4c5fad8531ea2d33f83fc
8,0,Men,BLR,Qualification,1st,Litvinovich Ivan,7680a87402e897bb8b0357190e27f365
9,0,Men,JPN,Qualification,1st,Kishi Daiki,c0fa36e3c33646804600fa318e2c70c2


In [88]:
test = df["Name"].iloc[0].split()
test = test[0].capitalize() + " " + test[1].capitalize()
test

'Page Bryony'

In [100]:
test_name

'H A'